<a href="https://colab.research.google.com/github/saivarshitnune/NLP_Projects/blob/Varshith/Update_ques_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Any
from api.main.config import settings

from api.initializer import (redis_db_instance, logger_instance, firestore_db_instance)
from api.main.utility.helpers import embeddings_helper

logger = logger_instance.get_logger(_name_)

In [ ]:
class UpdateQuestionPhrasesService(object):
    """This class consists of Services method required by /add_questions route"""

    def __init__(self) -> None:
        pass

    @staticmethod
    async def update_phrases(list_of_questions: list) -> Any:
        """This function takes input of list of questions whose phrases has to be updated with sent mid"""

        logger.info('Received %s Questions', len(list_of_questions))

        # loop through single questions
        result = False
        for index, master_question in enumerate(list_of_questions):
            master_question = dict(master_question)

            # get given MID
            mid = master_question['mid']

            if mid is None:
              given_uuids = [dict(uuids)['uuid'] for uuids in master_question['uuids']]
              for i in given_uuids:
               document = await firestore_db_instance.get_documents(settings.ALL_FAQS_COLLECTION_NAME, i)
               if document['Status'] != "Trained":
                 print("ISSUE Here:" + str(i))
                 continue
              print("no uuid present as not-trained")
              result = True
              continue

            else:
               # get the question digest
               question_digest = master_question['question_digest']
               # get the phrases of the given question digest
               digest_phrases_from_firestore, uuid_attribute_mapping = await firestore_db_instance.get_phrases_from(question_digest )
               if digest_phrases_from_firestore is not None:
                 # extracts embeddings of selected questions phrases
                 input_phrase_list = digest_phrases_from_firestore['phrases'].tolist()
                 digest_phrases_from_firestore["extracted_embeddings"] = embeddings_helper.extract_embeddings(
                input_phrase_list )

                 master_question["uuids"] = uuid_attribute_mapping

                 print("The length of phrases for MID %s is %s", mid, len(input_phrase_list))

                 store = await embeddings_helper.store_question_embeddings(
            digest_phrases_from_firestore, mid, master_question, True )

                 result = True

               else:
                 print("MID %s didnt get updated (no phrases present)" + mid)
                 return False

            return result

      @staticmethod
      async def update_question_new(list_of_questions: list) -> Any:
    """This function takes input of list of questions whose phrases has to be remove from old mid and"""
     logger.info('Received %s Questions', len(list_of_questions))
     result = False

     # loop through single questions
     for index, master_question in enumerate(list_of_questions):
        master_question = dict(master_question)

        new_question_digest = master_question['question_digest']
        old_question_digest = master_question['old_question_digest']

        print("new question digest:", new_question_digest)
        print("old question digest:", old_question_digest)

        new_digest_phrases_from_firestore, new_uuid_attribute_mapping = await firestore_db_instance.get_phrases_from_question-digest(
            new_question_digest
        )
        old_digest_phrases_from_firestore, old_uuid_attribute_mapping = await firestore_db_instance.get_phrases_from_question_digest(
            old_question_digest
        )

        print("new digest_phrases_from_firestore:", new_digest_phrases_from_firestore)
        print("old digest_phrases_from_firestore:", old_digest_phrases_from_firestore)

        # Extracts embeddings of selected questions phrase
        input_phrase_list = new_digest_phrases_from_firestore['phrases'].tolist()
        new_digest_phrases_from_firestore["extracted_embeddings"] = embeddings_helper.extract_embeddings(input_phrase_list)

       # Get given MID
       old_mid = master_question['old_mid']

      # Generate new MID
       new_mid = settings.MODEL_EMBEDDINGS_PREFIX + str(redis_db_instance.increment_mid(settings.MODEL_EMBEDDINGS_PREFIX))

       master_question["uuids"] = new_uuid_attribute_mapping

     # Store updated embeddings of new MID
     print("The length of phrases for new MID %s is %s", new_mid, len(input_phrase_list))
     await embeddings_helper.store_question_embeddings(new_digest_phrases_from_firestore, new_mid, master_question, True)

     if old_digest_phrases_from_firestore is not None:
       # Print a message indicating that the question was not updated
       print("Question was not updated")
       # Extract embeddings for the old digest phrases
       input_phrase_list = old_digest_phrases_from_firestore['phrases'].tolist()
       old_digest_phrases_from_firestore["extracted_embeddings"] = embeddings_helper.extract_embeddings(input_phrase_list)

      # Get old MID master question
      old_mid_master_question = redis_db_instance.get_key_value(old_mid, "question")

      # Create old master question dictionary
        old_master_question_dictionary = {
        "question": old_mid_master_question,
        "question_digest": old_question_digest,
        "uuids": old_uuid_attribute_mapping}

     # Store updated embeddings of old MID
     print("The length of phrases for old MID %s is %s", old_mid, len(input_phrase_list))
     await embeddings_helper.store_question_embeddings(
     old_digest_phrases_from_firestore,
     old_mid,
     old_master_question_dictionary,
     True)
     result = True

   else:
    logger.info('QuestionDigest %s empty, deleting the MID %s', old_question_digest, old_mid)
    await embeddings_helper.delete_mid(old_mid)
    await firestore_db_instance.delete_document(settings.FAQS_UUID_MAPPING_COLLECTION_NAME, old_question_digest)
    result = True

 return result